# Competição DSA de Machine Learning - Dezembro 2019
# Testes com H2O AutoML

# Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import h2o
import h2o.automl
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

# Inicializando o cluster
h2o.init()
h2o.cluster().show_status()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_211"; Java(TM) SE Runtime Environment (build 1.8.0_211-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from /Users/rodrigolima82/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/cl/573sf10x5pq298_1bby46zzc0000gn/T/tmp0ybevoth
  JVM stdout: /var/folders/cl/573sf10x5pq298_1bby46zzc0000gn/T/tmp0ybevoth/h2o_rodrigolima82_started_from_python.out
  JVM stderr: /var/folders/cl/573sf10x5pq298_1bby46zzc0000gn/T/tmp0ybevoth/h2o_rodrigolima82_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Sao_Paulo
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_rodrigolima82_0uog9v
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Sao_Paulo
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_rodrigolima82_0uog9v
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


# Importando dataset 

In [2]:
train = pd.read_csv('../dataset/dataset_treino.csv')
test = pd.read_csv('../dataset/dataset_teste.csv')
test_leak = pd.read_csv('../dataset/teste_leakage.csv')

# Juntando os datasets de treino e teste para melhorar a validacao
train = train.append(test_leak)

/Users/rodrigolima82/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [5]:
# Transformando as features categorias com LabelEncoder
le = LabelEncoder()

for i, col in enumerate(train):
    if train[col].dtype == 'object':
        train[col] = le.fit_transform(np.array(train[col].astype(str)).reshape((-1,)))

In [6]:
# Realizando tratamento de missing value
for c in train.columns:
    if c != 'ID' and c != 'target':
        train[c].fillna(train[c].mean(),inplace=True)

In [8]:
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures

# Add decomposed components: PCA / ICA etc.
n_comp = 4

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=42)
tsvd_results_df = tsvd.fit_transform(train.drop(columns = ['ID','target'], axis = 1))

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_df = pca.fit_transform(train.drop(columns = ['ID','target'], axis = 1))

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_df = ica.fit_transform(train.drop(columns = ['ID','target'], axis = 1))

# GRP
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=42)
grp_results_df = grp.fit_transform(train.drop(columns = ['ID','target'], axis = 1))

# SRP
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=42)
srp_results_df = srp.fit_transform(train.drop(columns = ['ID','target'], axis = 1))

In [9]:
# Append decomposition components to datasets
for i in range(1, n_comp+1):
    train['pca_' + str(i)]  = pca2_results_df[:,i-1]
    train['ica_' + str(i)]  = ica2_results_df[:,i-1]
    train['tsvd_' + str(i)] = tsvd_results_df[:,i-1]
    train['grp_' + str(i)]  = grp_results_df[:,i-1]
    train['srp_' + str(i)]  = srp_results_df[:,i-1] 

In [ ]:
# Convertendo os datasets em H2OFrame
train = df[df['target'].notnull()]
test = df[df['target'].isnull()]

train.shape, test.shape

In [10]:
hf_train = h2o.H2OFrame(train)
#hf_test = h2o.H2OFrame(test)

#hf_train.shape, hf_test.shape

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [11]:
# Convertendo coluna target para fator
hf_train['target'] = hf_train['target'].asfactor()

# Funcao para converter colunas numericas para fator
hf_train['v38'] = hf_train['v38'].asfactor()
#hf_test['v38'] = hf_test['v38'].asfactor()

hf_train['v62'] = hf_train['v62'].asfactor()
#hf_test['v62'] = hf_test['v62'].asfactor()

hf_train['v72'] = hf_train['v72'].asfactor()
#hf_test['v72'] = hf_test['v72'].asfactor()

hf_train['v129'] = hf_train['v129'].asfactor()
#hf_test['v129'] = hf_test['v129'].asfactor()

hf_train['v107'] = hf_train['v107'].asfactor()
#hf_test['v107'] = hf_test['v107'].asfactor()

hf_train['v110'] = hf_train['v110'].asfactor()
#hf_test['v110'] = hf_test['v110'].asfactor()

hf_train['v112'] = hf_train['v112'].asfactor()
#hf_test['v112'] = hf_test['v112'].asfactor()

hf_train['v125'] = hf_train['v125'].asfactor()
#hf_test['v125'] = hf_test['v125'].asfactor()

hf_train['v24'] = hf_train['v24'].asfactor()
#hf_test['v24'] = hf_test['v24'].asfactor()

hf_train['v3'] = hf_train['v3'].asfactor()
#hf_test['v3'] = hf_test['v3'].asfactor()

hf_train['v30'] = hf_train['v30'].asfactor()
#hf_test['v30'] = hf_test['v30'].asfactor()

hf_train['v31'] = hf_train['v31'].asfactor()
#hf_test['v31'] = hf_test['v31'].asfactor()

hf_train['v47'] = hf_train['v47'].asfactor()
#hf_test['v47'] = hf_test['v47'].asfactor()

hf_train['v52'] = hf_train['v52'].asfactor()
#hf_test['v52'] = hf_test['v52'].asfactor()

hf_train['v66'] = hf_train['v66'].asfactor()
#hf_test['v66'] = hf_test['v66'].asfactor()

hf_train['v74'] = hf_train['v74'].asfactor()
#hf_test['v74'] = hf_test['v74'].asfactor()

hf_train['v75'] = hf_train['v75'].asfactor()
#hf_test['v75'] = hf_test['v75'].asfactor()

hf_train['v79'] = hf_train['v79'].asfactor()
#hf_test['v79'] = hf_test['v79'].asfactor()

hf_train['v91'] = hf_train['v91'].asfactor()
#hf_test['v91'] = hf_test['v91'].asfactor()

# Usando H2O PCA

# Usando o H2O Gradiente Boosting Estimator

In [15]:
hf_train.columns[3:]

['v1',
 'v10',
 'v100',
 'v101',
 'v102',
 'v103',
 'v104',
 'v105',
 'v106',
 'v107',
 'v108',
 'v109',
 'v11',
 'v110',
 'v111',
 'v112',
 'v113',
 'v114',
 'v115',
 'v116',
 'v117',
 'v118',
 'v119',
 'v12',
 'v120',
 'v121',
 'v122',
 'v123',
 'v124',
 'v125',
 'v126',
 'v127',
 'v128',
 'v129',
 'v13',
 'v130',
 'v131',
 'v14',
 'v15',
 'v16',
 'v17',
 'v18',
 'v19',
 'v2',
 'v20',
 'v21',
 'v22',
 'v23',
 'v24',
 'v25',
 'v26',
 'v27',
 'v28',
 'v29',
 'v3',
 'v30',
 'v31',
 'v32',
 'v33',
 'v34',
 'v35',
 'v36',
 'v37',
 'v38',
 'v39',
 'v4',
 'v40',
 'v41',
 'v42',
 'v43',
 'v44',
 'v45',
 'v46',
 'v47',
 'v48',
 'v49',
 'v5',
 'v50',
 'v51',
 'v52',
 'v53',
 'v54',
 'v55',
 'v56',
 'v57',
 'v58',
 'v59',
 'v6',
 'v60',
 'v61',
 'v62',
 'v63',
 'v64',
 'v65',
 'v66',
 'v67',
 'v68',
 'v69',
 'v7',
 'v70',
 'v71',
 'v72',
 'v73',
 'v74',
 'v75',
 'v76',
 'v77',
 'v78',
 'v79',
 'v8',
 'v80',
 'v81',
 'v82',
 'v83',
 'v84',
 'v85',
 'v86',
 'v87',
 'v88',
 'v89',
 'v9',
 'v90',
 

In [16]:
# Indicando as colunas preditoras e a target
predictors = hf_train.columns[3:]
target = 'target'

# Split em treino e teste
X_train, X_valid = hf_train.split_frame(ratios = [.9], seed = 42)

# Usando o balanceamento de classes
gbm = H2OGradientBoostingEstimator(balance_classes=True, nfolds = 5, seed=42)
gbm.train(x=predictors, y=target, training_frame=X_train, validation_frame=X_valid)

print('logloss', gbm.logloss(valid = True))

# grid over - balance_classes
# import Grid Search
from h2o.grid.grid_search import H2OGridSearch

# Selecionando os valores para 'balance_classes' do grid
hyper_params = {'balance_classes': [True, False]}

# Inicializando o estimador GBM
gbm_2 = H2OGradientBoostingEstimator(seed = 1234)

# Construindo um grid search com os hyperparametros do GBM
grid = H2OGridSearch(model = gbm_2, hyper_params = hyper_params, search_criteria = {'strategy': "Cartesian"})

# Treino do modelo usando Grid
grid.train(x = predictors, y = target, training_frame = X_train, validation_frame = X_valid)

# Ordenar o grid models por logloss
sorted_grid = grid.get_grid(sort_by='logloss', decreasing=False)
print(sorted_grid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
logloss 0.49443183979800254
gbm Grid Build progress: |████████████████████████████████████████████████| 100%
    balance_classes  \
0             false   
1              true   

                                                        model_ids  \
0  Grid_GBM_py_32_sid_aaed_model_python_1577045172929_122_model_2   
1  Grid_GBM_py_32_sid_aaed_model_python_1577045172929_122_model_1   

               logloss  
0  0.49414838646819115  
1   0.4944108857371875  



# Agora com H2OAutoML

In [35]:
from h2o.automl import H2OAutoML

# Indicando as colunas preditoras e a target
predictors = hf_train.columns[3:]
target = 'target'

# Split em treino e teste
X_train, X_valid = hf_train.split_frame(ratios = [.8], seed = 42)

# Executa o AutoML para 20 modelos (limitado a 2h de duracao máxima)
aml = H2OAutoML(max_models=20, max_runtime_secs=60*60*60*60, seed=1)
#aml = H2OAutoML(max_models=20, seed=1)

aml.train(x=predictors, y=target, training_frame=X_train, validation_frame=X_valid)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [36]:
# Detalhes do melhor modelo
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20191222_194429

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.036161589467755974
RMSE: 0.19016200847634096
LogLoss: 0.18271109513879316
Null degrees of freedom: 112116
Residual degrees of freedom: 112108
Null deviance: 125225.45550576613
Residual deviance: 40970.03970735215
AIC: 40988.03970735215
AUC: 0.9993465932219487
pr_auc: 0.9966978767149256
Gini: 0.9986931864438975

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6212808982919231: 


,,0,1,Error,Rate
0,0,26991.0,646.0,0.0234,(646.0/27637.0)
1,1,652.0,83828.0,0.0077,(652.0/84480.0)
2,Total,27643.0,84474.0,0.0116,(1298.0/112117.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.621281,0.992317,186.0
1,max f2,0.551568,0.994539,205.0
2,max f0point5,0.676820,0.994171,169.0
3,max accuracy,0.621281,0.988423,186.0
4,max precision,0.936565,1.000000,0.0
5,max recall,0.358117,1.000000,264.0
6,max specificity,0.936565,1.000000,0.0
7,max absolute_mcc,0.621281,0.968837,186.0
8,max min_per_class_accuracy,0.656443,0.986790,176.0
9,max mean_per_class_accuracy,0.665606,0.987154,173.0



Gains/Lift Table: Avg response rate: 75.35 %, avg score: 71.74 %


...

,,0,1,Error,Rate
0,0,5060.0,22577.0,0.8169,(22577.0/27637.0)
1,1,2221.0,82259.0,0.0263,(2221.0/84480.0)
2,Total,7281.0,104836.0,0.2212,(24798.0/112117.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.396309,0.869013,303.0
1,max f2,0.166835,0.939870,371.0
2,max f0point5,0.716652,0.837073,163.0
3,max accuracy,0.456807,0.780230,282.0
4,max precision,0.934929,1.000000,0.0
5,max recall,0.048200,1.000000,399.0
6,max specificity,0.934929,1.000000,0.0
7,max absolute_mcc,0.754167,0.341382,139.0
8,max min_per_class_accuracy,0.780215,0.690451,122.0
9,max mean_per_class_accuracy,0.794994,0.693387,112.0



Gains/Lift Table: Avg response rate: 75.35 %, avg score: 75.35 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010007,0.930105,1.309400,1.309400,0.986631,0.932429,0.986631,0.932429,0.013104,0.013104,30.939997,30.939997
1,,2,0.020006,0.927290,1.293994,1.301700,0.975022,0.928622,0.980829,0.930526,0.012938,0.026042,29.399355,30.170020
2,,3,0.030004,0.925105,1.303465,1.302288,0.982159,0.926171,0.981272,0.929075,0.013033,0.039074,30.346469,30.228819
3,,4,0.040003,0.923191,1.286890,1.298440,0.969670,0.924140,0.978372,0.927842,0.012867,0.051941,28.689020,29.843955
4,,5,0.050001,0.921338,1.286890,1.296130,0.969670,0.922266,0.976632,0.926727,0.012867,0.064808,28.689020,29.613009
5,,6,0.100003,0.912821,1.274587,1.285359,0.960400,0.917054,0.968516,0.921890,0.063731,0.128539,27.458711,28.535860
6,,7,0.150004,0.904366,1.244048,1.271588,0.937389,0.908623,0.958140,0.917468,0.062204,0.190743,24.404815,27.158845
7,,8,0.200005,0.895336,1.226293,1.260265,0.924010,0.899916,0.949608,0.913080,0.061316,0.252060,22.629294,26.026457
8,,9,0.299999,0.875606,1.188876,1.236470,0.895817,0.885748,0.931678,0.903970,0.118880,0.370940,18.887631,23.646990
9,,10,0.400002,0.851593,1.142725,1.213033,0.861042,0.864045,0.914019,0.893988,0.114276,0.485215,14.272511,21.303318


# Previsoes

In [23]:
# Transformando as features categorias com LabelEncoder
for i, col in enumerate(test):
    if test[col].dtype == 'object':
        test[col] = le.fit_transform(np.array(test[col].astype(str)).reshape((-1,)))
        
# Realizando tratamento de missing value
for c in test.columns:
    if c != 'ID':
        test[c].fillna(test[c].mean(),inplace=True)

# Add decomposed components: PCA / ICA etc.
n_comp = 4

# tSVD
#tsvd = TruncatedSVD(n_components=n_comp, random_state=42)
tsvd_results_test = tsvd.fit_transform(test.drop(columns = ['ID'], axis = 1))

# PCA
#pca = PCA(n_components=n_comp, random_state=42)
pca2_results_test = pca.fit_transform(test.drop(columns = ['ID'], axis = 1))

# ICA
#ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_test = ica.fit_transform(test.drop(columns = ['ID'], axis = 1))

# GRP
#grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=42)
grp_results_test = grp.fit_transform(test.drop(columns = ['ID'], axis = 1))

# SRP
#srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=42)
srp_results_test = srp.fit_transform(test.drop(columns = ['ID'], axis = 1))


# Append decomposition components to datasets
for i in range(1, n_comp+1):
    test['pca_' + str(i)]  = pca2_results_test[:,i-1]
    test['ica_' + str(i)]  = ica2_results_test[:,i-1]
    test['tsvd_' + str(i)] = tsvd_results_test[:,i-1]
    test['grp_' + str(i)]  = grp_results_test[:,i-1]
    test['srp_' + str(i)]  = srp_results_test[:,i-1] 

In [24]:
hf_test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
# Gerar as previsoes no dataset de teste
preds = aml.leader.predict(hf_test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [30]:
# Salvar o resultado no dataset de teste
result = preds[:,2]
result

p1
0.344554
0.868317
0.796753
0.61508
0.765382
0.565762
0.875431
0.57058
0.891705
0.875126


In [32]:
hf_test['target'] = result

In [33]:
hf_test.head()

ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,pca_1,ica_1,tsvd_1,grp_1,srp_1,pca_2,ica_2,tsvd_2,grp_2,srp_2,pca_3,ica_3,tsvd_3,grp_3,srp_3,pca_4,ica_4,tsvd_4,grp_4,srp_4,target
0,1.37547,11.3611,2,4.20078,6.577,2.08178,1.78439,0.011094,9.52381,1.31291,16.8519,6.96117,3.10746,12.2301,1.77305,2.85714,3.49382,0.0968257,0.225481,17.9102,7.43781,3040,9.55831e-07,4,0.0691803,0.954152,4.38662,0.573017,6.98885,7,0,1.11524,2.08178,5.2285,8.17844,12.1832,0.833334,0,2.84277,11.7546,6.7658,10.855,1.2963,10.6134,9.41717,0.10048,8,12.7745,7.50929,0.118847,8.59075,2,17.0213,0.0443126,1.33829,6,2.60223,0.291971,9.03861,0.817843,14.9115,1,0.0914661,5.96795,15.6115,0,8.02974,15.9124,8.57143,11.2222,3,1,1.92901,1,3,2.14686,6.171,15.7143,15,1.90476,8.91873,8.75073,2.45353,0.892192,1.86046,-0.892193,7.90497,1.78439,0.105298,0.98898,1,0.658269,4.9293,2.99237,0.759955,5.94795,5,10.0135,0.817844,19.708,4.18679,1.87394,4.12902,1.70189,0.00453471,11.6881,1,1.50248,2.12171,0,3.56877,20,36,12.2467,11.4286,1.2766,4.7779,5.83333,-9.97495e-07,1.45985,3.26733,7.61905,1.81524,1.11227e-07,6,1.29237,3.90335,1.48593,0,2.33333,1.42857,6214.2,0.00162383,3040.47,-378.57,-90.5552,56.2303,0.00453381,29.5378,-972.271,-5109.24,40.7532,0.00358799,24.7121,-1787.37,57.5729,0.859416,-0.00488265,51.1333,494.86,-23.3884,0.344554
1,1.62862,7.43609,2,4.14127,8.7398,2.43588,2.48213,1.51225,9.03184,1.29103,15.4468,6.61829,3.80957,10.7608,2.08377,4.91543,3.82122,0.847644,0.222761,17.7712,5.89519,7094,1.09603,3,1.70609,1.87521,2.73845,5.10507,8.20745,0,0,1.62463,2.16615,8.38743,8.11957,13.3589,0.740637,4,1.25366,6.73757,7.18467,12.9316,2.21441,10.784,9.13591,1.63961,2,12.5507,8.02013,1.02707,7.20118,0,15.7098,1.25659,1.56296,80,4.07764,7.71177,10.5782,1.7137,14.5757,1,1.69494,6.32715,15.8406,2,9.29062,17.5439,9.45081,12.2706,3,5,2.43621,1,3,2.39808,7.308,13.3455,4,2.20811,7.28548,6.23446,2.18015,1.6095,2.81592,1.22214,10.174,1.92437,1.52059,0.966614,2,0.584669,5.4662,3.85644,0.668345,6.4579,7.62473,7.66304,1.24782,12.0801,6.85395,2.89556,5.30476,2.64813,1.08877,11.7728,3,2.14806,4.17434,1,3.3721,4,36,16.2849,10.5469,2.28979,8.29147,8.36602,3.1753,1.29607,2.74404,6.82322,3.55694,0.938867,72,1.6725,3.23789,2.02859,0,1.93241,1.73626,2160.2,0.00264126,7094.92,-773.604,-216.455,-15.5333,-0.00310771,70.4941,-2374.79,-12002.8,-26.6767,0.00113506,11.3575,-4305.3,70.3855,-12.5287,0.00257347,-12.1748,1139.12,1.21909,0.868317
2,-4.90341e-07,8.20153,2,4.54437,6.5501,1.55844,2.46753,0.00716367,7.14286,1.57549,15.1429,6.89425,1.89979,13.3062,1.30208,3.33333,3.17741,0.0706761,0.172391,17.5375,8.25345,8384,-2.13722e-07,1,0.0596196,2.85714,2.20779,0.327134,8.31169,2,0,0.974026,1.55844,7.95899,8.31169,12.1951,-5.24933e-07,0,6.70735,7.70132,6.88312,12.0779,3.42857,10.2157,7.9084,0.0821936,7,12.7708,7.79221,0.840731,9.88515,0,15,0.0201262,0.779222,86,5.45454,-8.47604e-07,9.45201,2.85714,15.5611,1,0.0837184,5.63366,15.7787,2,9.09091,13.2,6.66667,7.86059,3,1,1.4881,1,3,1.91029,5.71428,16.6667,2,1.42857,8.9523,8.74489,1.94805,0.779221,1.86046,0.779221,5.88623,5.71428,0.0438512,0.918734,2,0.552642,4.35098,3.16366,0.604966,6.23377,12.6316,10.2146,0.95238,19.8,6.53556,0.762963,4.91753,1.98186,0.00279355,11.747,3,1.18877,2.08882,0,3.11688,16,10,15.6227,13.3333,3.75,4.74057,12.6316,4.91066e-08,0.6,2.5,5.71429,1.97093,0.0141226,22,1.12872,5.84416,1.47589,0,1.26316,-6.38002e-07,870.201,-0.00425483,8384.65,-935.649,-227.701,-26.3806,0.0027154,36.6394,-2811.49,-14160.5,22.9056,0.000550551,-31.9652,-5123.94,29.9617,13.

# Submissions

In [34]:
# Gravar o arquivo de submissão ao Kaggle
h2o.export_file(hf_test[['ID', 'target']],'submission_h2o_ml_v1.1.3.csv')

Export File progress: |███████████████████████████████████████████████████| 100%


In [ ]:
# Close cluster
h2o.cluster().shutdown()